In [1]:
import json
from elasticsearch import Elasticsearch
from py2neo import Graph, Node, Relationship

In [2]:
client = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme' : 'http'}])
client.ping()

True

In [3]:
graph_db = Graph("bolt://localhost:7687")

In [4]:
searchBody = {
  "size": 100,
  "_source": True,
  "query": {
    "match_all": {}
  }
}

result = client.search(index="train", body=searchBody)
trains = result['hits']['hits']
len(trains)

20

In [9]:
i = 1
for train in trains:
    try:
        print(f'{i})', end='')
        TrainNode = Node("Train",train_number=train['_source']['train_number'])
        graph_db.create(TrainNode)
        searchBody = {
            "size": 100,
            "_source": True,
            "query": {
                "term": {
                    "train_id": train['_id']
                }
            }
        }
        result = client.search(index="ticket", body=searchBody)
        tickets = result['hits']['hits']
        print(f'{len(tickets)} tickets')
        user_nodes = {}
        for ticket in tickets:
            if ticket['_source']['user_id'] not in user_nodes:
                UserNode = Node("Passenger",passenger_id=ticket['_source']['user_id'], personal=ticket['_source']['personal'])
                graph_db.create(UserNode)
                user_nodes[ticket['_source']['user_id']] = UserNode
            node = user_nodes[ticket['_source']['user_id']]
            # NodesRelationship = Relationship(node, "buy_ticket",  TrainNode, date_purchase=ticket['_source']['date_purchase'], price=ticket['_source']['price'])
            # graph_db.create(NodesRelationship)
            relation_query = """
            MATCH (passenger:Passenger), (train:Train)
            WHERE passenger.passenger_id = $passenger_id AND train.train_number = $train_number
            CREATE (passenger)-[:buy_ticket {date_purchase: $date_purchase, price: $price}]->(train)
            """
            parameters = {
                "passenger_id": ticket['_source']['user_id'],
                "train_number": train['_source']['train_number'],
                "date_purchase": ticket['_source']['date_purchase'],
                "price": ticket['_source']['price']
            }
            graph_db.run(relation_query, parameters)
        i += 1
    except Exception as e:
        print(f'Exception: {e}')
        continue

1)2 tickets
2)6 tickets
3)1 tickets
4)2 tickets
5)1 tickets
6)2 tickets
7)1 tickets
8)0 tickets
9)0 tickets
10)3 tickets
11)1 tickets
12)1 tickets
13)0 tickets
14)2 tickets
15)0 tickets
16)1 tickets
17)0 tickets
18)5 tickets
19)1 tickets
20)1 tickets


##### Удалить все записи в БД
```
MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n,r
```

##### Выбрать поезда, пассажиры и связи между ними
```
MATCH (n) RETURN n
```

##### Найти поезд, на который было приобретено больше всего билетов
```
MATCH (t:Train)<-[r:`buy_ticket`]-() WITH t, count(r) as cnt ORDER BY cnt DESC RETURN t.train_number, cnt LIMIT 1
```